# Clustering IMDB movie actors

Defining notebook params:

In [ ]:
seed = 42
n_spectral_components = 2^10
min_cluster_size = 300
min_samples = 300
pdf_postfix = "_4"

Import stuff and ignore warnings:

In [ ]:
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
from collections import Counter
from hdbscan import HDBSCAN
from matplotlib.axes._axes import _log
_log.setLevel("ERROR")
from matplotlib.cm import get_cmap
from matplotlib.patches import Patch
from sklearn.manifold import spectral_embedding
from umap import UMAP

%matplotlib inline

Loading the network and taking the largest connected component:

In [ ]:
with open("data/imdb.pkl", "rb") as f:
    G = nx.Graph()
    G.add_edges_from(pickle.load(f))

G.remove_nodes_from(set(nx.nodes(G)) - max(nx.connected_components(G), key=len))
adj_matrix = nx.to_numpy_array(G)
node_list = list(G.nodes())

print(f"nodes: {len(node_list)}")
print(f"edges: {len(G.edges())}")

Using Spectral Embedding, reducing dimensions with UMAP and clustering with HDBSCAN:

In [ ]:
np.random.seed(seed)

embedding = spectral_embedding(
    adj_matrix, n_components=n_spectral_components, drop_first=False, random_state=np.random.RandomState(seed)
)
embedding = UMAP(
    n_components=2, n_neighbors=30, min_dist=0.0, random_state=np.random.RandomState(seed)
).fit_transform(embedding)
clusters = HDBSCAN(
    min_samples=min_samples, min_cluster_size=min_cluster_size, core_dist_n_jobs=-1
).fit_predict(embedding)

counter = Counter(clusters)
print(f"clusters: {np.amax(clusters) + 1}")
print(f"cluster sizes: {[y for y in sorted(counter.items(), key=lambda x: x[1], reverse=True) if y[0] != -1]}")
print(f"noise level: {np.round(counter[-1] / len(clusters), 3)}")

Defining some helper functions for printing leaderboards:

In [ ]:
def display_side_by_side(dfs, captions):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

def print_leaderboards(df):
    df_list = []
    caption_list = []
    for i in range(df["cluster"].max()):
        caption_list.append(f"Cluster idx: {i}")
        cluster_top_actors = df_actor_info[df_actor_info["cluster"] == i].reset_index()
        cluster_top_actors = cluster_top_actors.drop(["cluster", "index", "nconst"], axis=1)
        df_list.append(cluster_top_actors.head())
    display_side_by_side(df_list, caption_list)    

d = {"nconst": node_list, "cluster": clusters}
df_actor_info = pd.DataFrame(d)
df_actor_info["degree"] = df_actor_info["nconst"].apply(G.degree)

betweenness = nx.betweenness_centrality(G)
df_betweenness = pd.DataFrame(betweenness.items(), columns=["nconst", "betweenness"])
df_actor_info = df_actor_info.merge(df_betweenness, how="inner")

df_names = pd.read_csv("data/actor_names.csv", sep=",").reset_index()
df_names = df_names.drop("index", axis=1)
df_actor_info = df_actor_info.merge(df_names, how="inner")

Printing leaderboards for each cluster when sorted by degree:

In [ ]:
df_actor_info = df_actor_info.sort_values(by="degree", ascending=False)
print_leaderboards(df_actor_info)

Printing leaderboards for each cluster when sorted by betweenness:

In [ ]:
df_actor_info = df_actor_info.sort_values(by="betweenness", ascending=False)
print_leaderboards(df_actor_info)

Visualizing the network using Spring and Custom layouts:

In [ ]:
np.random.seed(seed)

def draw_network(G, pos, patch_list, node_colors, pdf_name, node_size=1, edge_width=.1, font_size=1):
    nx.draw_networkx_nodes(G, pos=pos, node_size=node_size, node_color=node_colors)
    nx.draw_networkx_edges(G, pos=pos, width=edge_width)
    # nx.draw_networkx_labels(G, pos=pos, font_size=font_size)
    plt.legend(handles=patch_list, loc="center left", bbox_to_anchor=(1, 0.5), ncol=2, fancybox=True)
    plt.savefig(f"results/{pdf_name + pdf_postfix}.pdf", bbox_inches="tight")
    plt.show()

cmap = get_cmap("tab20").colors
cmap = [cmap[i] for i in range(len(cmap)) if i % 2 == 0] + [cmap[i] for i in range(len(cmap)) if i % 2 == 1]
node_colors = [cmap[cluster] for cluster in clusters]
patch_list = [Patch(color=cmap[i], label=i) for i in np.arange(-1, np.amax(clusters) + 1)]

pos_sp = nx.spring_layout(G, seed=seed)
pos_su = {node_list[i]: embedding[i] for i in range(len(node_list))}

draw_network(G, pos=pos_sp, patch_list=patch_list, node_colors=node_colors, pdf_name="graph_sp")
draw_network(G, pos=pos_su, patch_list=patch_list, node_colors=node_colors, pdf_name="graph_su")

Removing nodes labeled as noise by HDBSCAN and visualizing the results:

In [ ]:
# G.remove_nodes_from([node_list[i] for i in range(len(node_list)) if clusters[i] == -1])
# node_colors = [cmap[cluster] for cluster in clusters if cluster != -1]
# 
# draw_network(G, pos=pos_sp, cmap=cmap, node_colors=node_colors, pdf_name="graph_sp_clean")
# draw_network(G, pos=pos_su, cmap=cmap, node_colors=node_colors, pdf_name="graph_su_clean")